# CNN (new)

In [ ]:
%matplotlib inline 
import numpy as np
import pandas as pd


import sklearn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras import layers, regularizers,Model, utils
from tensorflow.keras.callbacks import TensorBoard,EarlyStopping
%load_ext tensorboard

import plotly.express as px
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import widgets

from datetime import datetime,time
from pathlib import Path
import pickle

from livelossplot import PlotLossesKeras

## Load Data

In [ ]:
dbfile = open('otherLargeFiles/CNN-dataset.pkl', 'rb')     
dataset = pickle.load(dbfile)
dbfile.close()

## Split into training/validation

In [ ]:
longitudeLabels = dataset["labels"][:,0]
latitudeLabels = dataset["labels"][:,1]

combinedLabels = np.concatenate((np.reshape(longitudeLabels, (-1,1)), np.reshape(latitudeLabels, (-1,1))), axis=1)
trainInputs, testInputs, trainLabels, testLabels = train_test_split(dataset["samples"], combinedLabels, test_size=0.5, shuffle=False)
trainInputs, valInputs, trainLabels, valLabels = train_test_split(trainInputs, trainLabels, test_size=0.15, shuffle=False)

## Construct

In [ ]:
#specify input dimensionality
outputDimension = 2
sequenceLength = trainInputs.shape[1]

#construct our neural network
convActivation = "relu"
kernelSize = 9
batchSize = 64
epochs = 10

def convolutionalLayer(size, name):
    return layers.Conv1DTranspose(size, kernel_size=kernelSize, activation=convActivation, name=name, kernel_regularizer=regularizers.l2(1e-5), padding="same")

inputLayer = layers.Input(shape=(sequenceLength, 13), name="input")
modelLayers = [inputLayer]

def layerByName(n):
    for layer in modelLayers:
        if layer.name.split("/")[0] == n:
            return layer
    return None


modelLayers.append(convolutionalLayer(16,"conv0")(modelLayers[-1]))
modelLayers.append(convolutionalLayer(8,"conv1")(modelLayers[-1]))
modelLayers.append(convolutionalLayer(4,"conv2")(modelLayers[-1]))
modelLayers.append(convolutionalLayer(2,"conv3")(modelLayers[-1]))
modelLayers.append(convolutionalLayer(2,"conv4")(modelLayers[-1]))
modelLayers.append(layers.Concatenate(name="concat0")([layerByName("conv4"), layerByName("conv3")]))
modelLayers.append(convolutionalLayer(4,"conv5")(modelLayers[-1]))
modelLayers.append(layers.Concatenate(name="concat1")([layerByName("conv5"), layerByName("conv2")]))
modelLayers.append(convolutionalLayer(8,"conv6")(modelLayers[-1]))
modelLayers.append(layers.Concatenate(name="concat2")([layerByName("conv6"), layerByName("conv1")]))
modelLayers.append(convolutionalLayer(16,"conv7")(modelLayers[-1]))
modelLayers.append(layers.Concatenate(name="concat3")([layerByName("conv7"), layerByName("conv0")]))
modelLayers.append(layers.Dense(32, activation='sigmoid', name="dense1", kernel_regularizer=regularizers.l2(1e-5))(modelLayers[-1]))
modelLayers.append(layers.Flatten(name="flatten1")(modelLayers[-1]))
outputLayer = layers.Dense(outputDimension, activation='sigmoid', name="output")(modelLayers[-1])

## Compile

In [ ]:
combinedModel = Model(inputs=inputLayer, outputs=[outputLayer], name='combined_mlp')
combinedModel.compile(loss='mean_squared_error', metrics=[tf.keras.metrics.RootMeanSquaredError()], optimizer='adam')
combinedModel.save('model.h5')
combinedModel.summary()


## Train

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
combinedHistory = combinedModel.fit(trainInputs, trainLabels, epochs=epochs, batch_size=batchSize, callbacks=[PlotLossesKeras(),tensorboard_callback], validation_data=(valInputs, valLabels))

In [ ]:
pd.DataFrame({
    "min_val_loss": [],
    "hidden_layer_sizes": [],
    "activation": [],
    "kernel_size": [],
    "batch_size": [],
    "epochs": []
}).to_csv("trainingLogs.csv")

In [ ]:
trainingLogItem = {
    "min_val_loss": min(combinedHistory.history["val_loss"]),
    "hidden_layer_sizes": hiddenLayerSizes,
    "activation": convActivation,
    "kernel_size": kernelSize,
    "batch_size": batchSize,
    "epochs": epochs
}

trainingLogs = pd.read_csv("trainingLogs.csv")
trainingLogs.append(trainingLogItem, ignore_index=True)
trainingLogs.to_csv("trainingLogs.csv")

## Test

### Predict Test Set

In [ ]:
predictions = combinedModel.predict(testInputs)

scatterData = pd.DataFrame({
    "index":np.arange(predictions[:,0].shape[0]),
    "long_predicted": predictions[:,0].reshape(predictions[:,0].shape[0]),
    "long_actual": testLabels[:,0].reshape(testLabels[:,0].shape[0]),
    "lat_predicted": predictions[:,1].reshape(predictions[:,1].shape[0]),
    "lat_actual": testLabels[:,1].reshape(testLabels[:,1].shape[0])
})

fig = go.Figure()
fig.add_trace(go.Scatter(x=scatterData.index, y=scatterData.long_predicted, name="long_predicted"))
fig.add_trace(go.Scatter(x=scatterData.index, y=scatterData.long_actual, name="long_actual"))
fig.add_trace(go.Scatter(x=scatterData.index, y=scatterData.lat_predicted, name="lat_predicted"))
fig.add_trace(go.Scatter(x=scatterData.index, y=scatterData.lat_actual, name="lat_actual"))

### Predict Training Set

In [ ]:
predictions = combinedModel.predict(trainInputs)

scatterData = pd.DataFrame({
    "index":np.arange(predictions[:,0].shape[0]),
    "long_predicted": predictions[:,0].reshape(predictions[:,0].shape[0]),
    "long_actual": trainLabels[:,0].reshape(trainLabels[:,0].shape[0]),
    "lat_predicted": predictions[:,1].reshape(predictions[:,1].shape[0]),
    "lat_actual": trainLabels[:,1].reshape(trainLabels[:,1].shape[0])
})

fig = go.Figure()
fig.add_trace(go.Scatter(x=scatterData.index, y=scatterData.long_predicted, name="long_predicted"))
fig.add_trace(go.Scatter(x=scatterData.index, y=scatterData.long_actual, name="long_actual"))
fig.add_trace(go.Scatter(x=scatterData.index, y=scatterData.lat_predicted, name="lat_predicted"))
fig.add_trace(go.Scatter(x=scatterData.index, y=scatterData.lat_actual, name="lat_actual"))